In [ ]:
import io
import pandas as pd
import numpy as np
from google.colab import files
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from statsmodels.graphics.api import abline_plot
from sklearn.metrics import roc_curve

In [ ]:
uploaded = files.upload()

Saving data_r.csv to data_r (1).csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded.get('data_r.csv')))
df

,jobTitle,gender,age,perfEval,edu,dept,seniority,basePay,bonus
0,Graphic Designer,Female,18,5,College,Operations,2,42363,9938
1,Software Engineer,Male,21,5,College,Management,5,108476,11128
2,Warehouse Associate,Female,19,4,PhD,Administration,5,90208,9268
3,Software Engineer,Male,20,5,Masters,Sales,4,108080,10154
4,Graphic Designer,Male,26,5,Masters,Engineering,5,99464,9319
...,...,...,...,...,...,...,...,...,...
995,Marketing Associate,Female,61,1,High School,Administration,1,62644,3270
996,Data Scientist,Male,57,1,Masters,Sales,2,108977,3567
997,Financial Analyst,Male,48,1,High School,Operations,1,92347,2724
998,Financial Analyst,Male,65,2,High School,Administration,1,97376,2225


In [ ]:
df['age_bin'] = pd.cut(df['age'], 5, labels=[1,2,3,4,5])
df

,jobTitle,gender,age,perfEval,edu,dept,seniority,basePay,bonus,age_bin
0,Graphic Designer,Female,18,5,College,Operations,2,42363,9938,1
1,Software Engineer,Male,21,5,College,Management,5,108476,11128,1
2,Warehouse Associate,Female,19,4,PhD,Administration,5,90208,9268,1
3,Software Engineer,Male,20,5,Masters,Sales,4,108080,10154,1
4,Graphic Designer,Male,26,5,Masters,Engineering,5,99464,9319,1
...,...,...,...,...,...,...,...,...,...,...
995,Marketing Associate,Female,61,1,High School,Administration,1,62644,3270,5
996,Data Scientist,Male,57,1,Masters,Sales,2,108977,3567,5
997,Financial Analyst,Male,48,1,High School,Operations,1,92347,2724,4
998,Financial Analyst,Male,65,2,High School,Administration,1,97376,2225,5


In [ ]:
df['Total_Compensation'] = df['basePay'] + df['bonus']
# df

In [ ]:
df['log_basePay'] = np.log10(df['basePay'])
df['log_bonus'] = np.log10(df['bonus'])
df['log_total_compensation'] = np.log10(df['Total_Compensation'])
# df

In [ ]:
df['male'] = df['gender'].map({"Female":0,"Male":1})
# df

In [ ]:
df['jobTitle'] = df['jobTitle'].astype('category')
df['edu'] = df['edu'].astype('category')
df['dept'] = df['dept'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   jobTitle                1000 non-null   category
 1   gender                  1000 non-null   object  
 2   age                     1000 non-null   int64   
 3   perfEval                1000 non-null   int64   
 4   edu                     1000 non-null   category
 5   dept                    1000 non-null   category
 6   seniority               1000 non-null   int64   
 7   basePay                 1000 non-null   int64   
 8   bonus                   1000 non-null   int64   
 9   age_bin                 1000 non-null   category
 10  Total_Compensation      1000 non-null   int64   
 11  log_basePay             1000 non-null   float64 
 12  log_bonus               1000 non-null   float64 
 13  log_total_compensation  1000 non-null   float64 
 14  male                    1

In [ ]:
df.describe()

,age,perfEval,seniority,basePay,bonus,Total_Compensation,log_basePay,log_bonus,log_total_compensation,male
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,41.393000,3.037000,2.971000,94472.653000,6467.161000,100939.814000,4.958431,3.786945,4.989673,0.532000
std,14.294856,1.423959,1.395029,25337.493272,2004.377365,25156.599655,0.124694,0.149996,0.114626,0.499225
min,18.000000,1.000000,1.000000,34208.000000,1703.000000,40828.000000,4.534128,3.231215,4.610958,0.000000
25%,29.000000,2.000000,2.000000,76850.250000,4849.500000,83443.000000,4.885645,3.685697,4.921390,0.000000
50%,41.000000,3.000000,3.000000,93327.500000,6507.000000,100047.000000,4.970010,3.813381,5.000204,1.000000
75%,54.250000,4.000000,4.000000,111558.000000,8026.000000,117656.000000,5.047501,3.904499,5.070614,1.000000
max,65.000000,5.000000,5.000000,179726.000000,11293.000000,184010.000000,5.254611,4.052809,5.264841,1.000000


In [ ]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)

In [ ]:
formula1 = 'Total_Compensation ~ male'
logm1 = smf.glm(formula=formula1, data = df_train, family=sm.families.Binomial()).fit()
logm1.summary() 

/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:890: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:942: RuntimeWarning: divide by zero encountered in true_divide
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:942: RuntimeWarning: invalid value encountered in add
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:943: RuntimeWarning: divide by zero encountered in log
  n * np.log(1 - mu)) * var_weights


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:     Total_Compensation   No. Observations:                  750
Model:                            GLM   Df Residuals:                      748
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Wed, 17 Mar 2021   Deviance:                   7.1795e+09
Time:                        11:58:00   Pearson chi2:                 3.63e+28
No. Iterations:                     2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   6.495e+20   3.57e+06   1.82e+14      0.000     6.5e+20     6.5e+20
male        5.647e+19   4.91e+06   1.15e+13      0.000    5.65e+19    5.65e+19
==============================================================================
"""

In [ ]:
formula2 = 'Total_Compensation ~ male + age_bin + perfEval + edu'
logm2 = smf.glm(formula=formula2, data = df_train, family=sm.families.Binomial()).fit()
logm2.summary() 

/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:890: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:942: RuntimeWarning: divide by zero encountered in true_divide
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:942: RuntimeWarning: invalid value encountered in add
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:943: RuntimeWarning: divide by zero encountered in log
  n * np.log(1 - mu)) * var_weights


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:     Total_Compensation   No. Observations:                  750
Model:                            GLM   Df Residuals:                      740
Model Family:                Binomial   Df Model:                            9
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Wed, 17 Mar 2021   Deviance:                   7.1795e+09
Time:                        11:58:12   Pearson chi2:                 3.63e+28
No. Iterations:                     2                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           5.112e+20   8.96e+06    5.7e+13      0.000    5.11e+20    5.11e+20
age_bin[T.2]        5.478e+19   7.77e+06   7.05e+12      0.000    5.48e+19    5.48e+19
age_bin[T.3]        1.426e+20   7.69e+06   1.85e+13      0.000    1.43e+20    1.43e+20
age_bin[T.4]        1.707e+20   7.59e+06   2.25e+13      0.000    1.71e+20    1.71e+20
age_bin[T.5]        2.455e+20   7.39e+06   3.32e+13      0.000    2.45e+20    2.45e+20
edu[T.High School] -1.885e+19   6.99e+06   -2.7e+12      0.000   -1.89e+19   -1.89e+19
edu[T.Masters]      3.438e+19   7.08e+06   4.85e+12      0.000    3.44e+19    3.44e+19
edu[T.PhD]          5.961e+19   7.12e+06   8.38e+12      0.000    5.96e+19    5.96e+19
male                5.334e+19   4.95e+06   1.08e+13      0.000    5.33e+19    5.33e+19
perfEval           -1.498e+17   1.73e+06  -8.68e+10      0.000    -1.5e+17    -1.5e+17
======================================================================================
"""

In [ ]:
formula3 = 'Total_Compensation ~ male + age_bin + perfEval + edu + dept + jobTitle + seniority'
logm3 = smf.glm(formula=formula3, data = df_train, family=sm.families.Binomial()).fit()
logm3.summary() 

/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:890: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:942: RuntimeWarning: divide by zero encountered in true_divide
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:942: RuntimeWarning: invalid value encountered in add
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:943: RuntimeWarning: divide by zero encountered in log
  n * np.log(1 - mu)) * var_weights


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:     Total_Compensation   No. Observations:                  750
Model:                            GLM   Df Residuals:                      726
Model Family:                Binomial   Df Model:                           23
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Wed, 17 Mar 2021   Deviance:                   7.1795e+09
Time:                        11:58:50   Pearson chi2:                 3.63e+28
No. Iterations:                     2                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                        3.071e+20   1.35e+07   2.28e+13      0.000    3.07e+20    3.07e+20
age_bin[T.2]                     5.016e+19   7.83e+06   6.41e+12      0.000    5.02e+19    5.02e+19
age_bin[T.3]                     1.243e+20   7.78e+06    1.6e+13      0.000    1.24e+20    1.24e+20
age_bin[T.4]                     1.787e+20   7.64e+06   2.34e+13      0.000    1.79e+20    1.79e+20
age_bin[T.5]                     2.373e+20   7.46e+06   3.18e+13      0.000    2.37e+20    2.37e+20
edu[T.High School]              -5.731e+18   7.03e+06  -8.15e+11      0.000   -5.73e+18   -5.73e+18
edu[T.Masters]                   3.504e+19   7.11e+06   4.93e+12      0.000     3.5e+19     3.5e+19
edu[T.PhD]                       4.611e+19   7.14e+06   6.46e+12      0.000    4.61e+19    4.61e+19
dept[T.Engineering]              1.755e+19   7.83e+06   2.24e+12      0.000    1.75e+19    1.75e+19
dept[T.Management]               2.074e+19    7.9e+06   2.63e+12      0.000    2.07e+19    2.07e+19
dept[T.Operations]               -6.03e+18   7.74e+06  -7.79e+11      0.000   -6.03e+18   -6.03e+18
dept[T.Sales]                    4.167e+19   7.73e+06   5.39e+12      0.000    4.17e+19    4.17e+19
jobTitle[T.Driver]              -2.325e+19   1.13e+07  -2.07e+12      0.000   -2.33e+19   -2.33e+19
jobTitle[T.Financial Analyst]    2.594e+19   1.06e+07   2.46e+12      0.000    2.59e+19    2.59e+19
jobTitle[T.Graphic Designer]    -1.105e+19   1.05e+07  -1.06e+12      0.000   -1.11e+19   -1.11e+19
jobTitle[T.IT]                  -1.512e+19   1.09e+07  -1.39e+12      0.000   -1.51e+19   -1.51e+19
jobTitle[T.Manager]              2.089e+20   1.13e+07   1.85e+13      0.000    2.09e+20    2.09e+20
jobTitle[T.Marketing Associate] -1.104e+20   1.04e+07  -1.06e+13      0.000    -1.1e+20    -1.1e+20
jobTitle[T.Sales Associate]      7.693e+18   1.06e+07   7.23e+11      0.000    7.69e+18    7.69e+18
jobTitle[T.Software Engineer]    8.955e+19   1.07e+07   8.37e+12      0.000    8.96e+19    8.96e+19
jobTitle[T.Warehouse Associate]  3.926e+17   1.08e+07   3.63e+10      0.000    3.93e+17    3.93e+17
male                             -3.36e+18    5.5e+06  -6.11e+11      0.000   -3.36e+18   -3.36e+18
perfEval                         6.209e+18   1.74e+06   3.57e+12      0.000    6.21e+18    6.21e+18
seniority                        6.452e+19   1.77e+06   3.64e+13      0.000    6.45e+19    6.45e+19
===================================================================================================
"""